<a href="https://colab.research.google.com/github/daour3/Defi3/blob/passworGeneration/Defi3_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [61]:
ls drive/MyDrive/Defi-Password

[Errno 2] No such file or directory: 'drive/MyDrive/Defi-Password/'
/content/drive/MyDrive/Defi-Password


In [ ]:
cd drive/MyDrive/Defi-Password/

In [ ]:
import torch
from torch.autograd import Variable
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import numpy as np
import unicodedata,string,random
from torch.utils.data import Dataset, DataLoader

In [ ]:
# device we will use "GPU" or "CPU"
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
#read the letters in train datasets
with open("data/train.txt","r") as file:
  txt = file.read()
 
letters = sorted(list(set(txt)))
letters += ['<EOS>']
n_letters = len(letters)

In [ ]:
print(n_letters)

94


In [ ]:
# returns the index of a character in the list of chars
def charToIndex(c):
    if c in letters:
      return letters.index(c)
    return 0

In [ ]:
# Read a file and split into lines
def readLines(filename):
    lines = open(filename).read().strip().split('\n')
    return [line.strip('\\') for line in lines] # remove the \ from the end of passwords

In [ ]:
lines = readLines("train.txt")
max_len = max([len(w) for w in lines]) #get the maximum lenght of a password in the training set
print(max_len)

66


In [ ]:
# create a dataset of classe that we will use to train the model 
class PasswordsDataset(Dataset):
    
    def __init__(self, passwords):
        self.passwords = passwords
        
    def __len__(self):
        return len(self.passwords)
    
    # it will return an X Y fo each line of the dataset X will be the table of onehot encoded array describing each character of the line [0:n]
    # and Y will be the line from [1:n] + the end of word '\\'
    # example: x= 'abcd' y='bcd\\'
    def __getitem__(self, idx):
        x_str = self.passwords[idx].ljust(max_len, '\\')[:max_len]
        y_str = x_str[1:] + '\\'
        x = torch.zeros((max_len, n_letters))
        y = torch.zeros(max_len)
        for i, c in enumerate(x_str):
            x[i, charToIndex(c)] = 1
        for i, c in enumerate(y_str):
            y[i] = charToIndex(c)
            
        return x, y

In [ ]:
trainset = PasswordsDataset(lines)
train_loader = DataLoader(trainset, batch_size=256, shuffle=True)
print(len(train_loader))
#train_iter = train_loader.__iter__()
#X, Y = train_iter.next()
train_iter = iter(train_loader)
X, Y = next(train_iter)
print(X.size(), Y.size())


5291
torch.Size([256, 66, 94]) torch.Size([256, 66])


In [ ]:
# hyper parameters
input_size = n_letters
hidden_size = 60
output_size = n_letters
num_layers = 2
train_batch_size = 128

In [ ]:
class Model(nn.Module):
    def __init__(self, input_size, hidden_size, output_size, num_layers):
        super(Model, self).__init__()
        self.input_size = input_size
        self.hidden_size = hidden_size
        self.num_layers = num_layers
        self.lstm1 = nn.LSTM(input_size=input_size, hidden_size=hidden_size, num_layers=num_layers, batch_first=True)
        self.fc2 = nn.Linear(hidden_size, output_size)
        self.fc3 = nn.Linear(output_size, output_size)
        
    def forward(self, X, states):
        ht, ct = states
        batch_size = X.size(0)
        out, (ht, ct) = self.lstm1(X, (ht, ct))
        out = F.relu(self.fc2(out))
        out = self.fc3(out)
        return out, (ht, ct)

In [ ]:
model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
model = nn.DataParallel(model)
model = model.to(device)

In [ ]:
ht = torch.zeros((num_layers, train_batch_size, hidden_size)).to(device)
ct = torch.zeros((num_layers, train_batch_size, hidden_size)).to(device)

In [ ]:
lr = 0.005
step_size = len(train_loader) * 1
gamma = 0.95
print(step_size)

5291


In [ ]:
criterion = nn.CrossEntropyLoss(reduction='mean')
optimizer = optim.Adam(model.parameters(), lr=lr)
lr_scheduler = optim.lr_scheduler.StepLR(optimizer=optimizer, step_size=step_size, gamma=gamma)

In [ ]:
def generate_password(model, start='a', k=5):
  
    if start == max_length:
        return password
    
    with torch.no_grad():
        
        ht = torch.zeros((num_layers, 1, hidden_size)).to(device)
        ct = torch.zeros((num_layers, 1, hidden_size)).to(device)
        len = 0
        password = start
        # model preparation to predict the rest of the password based on `start`
        for c in start:
            X = torch.zeros((1, 1, n_letters)) # [batch_size, timestep, n_letters]
            X[0, 0, charToIndex(c)] = 1
            out, (ht, ct) = model(X, (ht, ct))
            len += 1
        vals, ids = torch.topk(out[0], k) # 0 -> first eg in a batch
        id = np.random.choice(ids.cpu().numpy()[0]) # 0 -> first...
        c = letters[id]
        vals, ids = torch.topk(out[0], k) # 0 -> first eg in a batch
        id = np.random.choice(ids.cpu().numpy()[0]) # 0 -> first...
        c = letters[id]
        
        while c != "\\" and len <= max_len-1:
            X = torch.zeros((1, 1, n_letters)) # [batch_size, timestep, n_letters]
            X[0, 0, charToIndex(c)] = 1
            out, (ht, ct) = model(X, (ht, ct))
            vals, ids = torch.topk(out[0], k) # 0 -> first eg in a batch
            id = np.random.choice(ids.cpu().numpy()[0]) # 0 -> first...
            c = letters[id]
            len += 1
            password += c
    
        if password[-1] != "\\":
            password += "\\"
    
    return password

In [ ]:
passwords_list = lines
def sampler(model, start='a', n=10, k=5, only_new=False):
    
    passwords = []
    cnt = 0
    while cnt <= n:
        password = generate_password(model=model, start=start, k=k)
        if only_new: 
            if password not in passwords_list and password not in passwords:
                passwords.append(password)
                cnt += 1
        else:
            if password not in passwords:
                passwords.append(password)
                cnt += 1
    passwords = [password[:-1] for password in passwords]
    
    return passwords

In [ ]:
epochs = 20

# if you are testing this do not run the next cell 
# this is only for training it gonna take hours to end 

In [62]:
from tqdm.notebook import tqdm

epoch_losses = []
epoch_lrs = []
iteration_losses = []
iteration_lrs = []

for epoch in tqdm(range(1, epochs+1), desc="Epochs"):
    epoch_loss = 0
    epoch_lr = 0
    
    for i, (X, Y) in tqdm(enumerate(train_loader, 1), total=len(train_loader), desc="Epoch-{}".format(epoch)):
        X, Y = X.to(device), Y.to(device)
        
        ht = torch.zeros((num_layers, X.size(0), hidden_size)).to(device)
        ct = torch.zeros((num_layers, X.size(0), hidden_size)).to(device)

        optimizer.zero_grad()
        Y_pred_logits, (ht, ct) = model(X, (ht, ct))
        Y_pred_logits = Y_pred_logits.transpose(1, 2) # Check Loss Doc: [N, d1, C] -> [N, C, d1]
        loss = criterion(Y_pred_logits, Y.long())
        loss.backward(retain_graph=True)
        optimizer.step()
        lr_scheduler.step()
        
        iteration_losses.append(loss.item())
        iteration_lrs.append(lr_scheduler.get_last_lr()[0])
        epoch_loss += loss.item()
        epoch_lr += lr_scheduler.get_last_lr()[0]
        
    epoch_loss /= len(train_loader)
    epoch_lr /= len(train_loader)
    epoch_losses.append(epoch_loss)
    epoch_lrs.append(epoch_lr)

    message = "Epoch:{}    Loss:{}    LR:{}".format(epoch, epoch_loss, epoch_lr)
    print(message)
    passwords = sampler(model, start='jo', n=10, k=10, only_new=False)
    print(passwords)
    torch.save(model.state_dict(), "password_gen_" + str(epoch) + ".pth")

Epochs:   0%|          | 0/3 [00:00<?, ?it/s]

Epoch-1:   0%|          | 0/5291 [00:00<?, ?it/s]

Epoch:1    Loss:4.304233941074761    LR:0.0033170708097987225


NameError: ignored

In [ ]:
ls model/

ls: cannot access 'model/': No such file or directory


In [ ]:
# we load the prevouis trained model
path = "password_gen_2.pth" # change this name if you retrained the model 
model = Model(input_size=input_size, hidden_size=hidden_size, output_size=output_size, num_layers=num_layers)
model = nn.DataParallel(model)
model = model.to(device)
#model.load_state_dict(torch.load(path,map_location=torch.device('cpu'))) 
model.load_state_dict(torch.load(path)) 

RuntimeError: ignored

In [ ]:
"""
this function will test the model over the data in eval.txt
we will take every password and pass its first letters and then we will see if 
the model predict the password , and we will do that the first time by giving it just 1 letter and the secon 2 letters and sow on till 5 letters
"""
from tqdm.notebook import tqdm

def test():
  test_passwords = []
  results = [0,0,0,0,0]
  with open("eval.txt") as f:
    test_passwords = f.readlines()
  for password in tqdm(test_passwords):
    for i in range(1,6): # to take 1 letter than 2 that 3 ..... till 5
      password_guesses = sampler(model, start=password[:i], n=10, k=5 , only_new=True)
      if password in password_guesses:
        results[i-1]+=1
  for i in range(5):
    results[i]/=len(test_passwords)
  return results

In [ ]:
results = test()

  0%|          | 0/1999 [00:00<?, ?it/s]

In [ ]:
print("if we give the model: ")
for i,res in enumerate(results):
  print("\t",i,"lettes it predict:",res*100,"% password correct.")